In [ ]:
# Install dependencies
!pip install -U ultralytics
!pip install opencv-python-headless
!pip install albumentations
!pip install labelImg


In [ ]:
import torch
from ultralytics import YOLO
import shutil
import pandas as pd


In [ ]:
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

In [ ]:
DATASET_PATH = "/home/muliaandiki/project/NutriPlate/preprocessing/data/Dietary App Segment.v1i.yolov8"
DATA_YAML = f"{DATASET_PATH}/data.yaml"

BASE_MODEL = "yolov8s.pt" 

In [ ]:
model = YOLO(BASE_MODEL)
model.info()

In [ ]:
model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=640,
    batch=4,              
    optimizer="AdamW",
    patience=20,          
    device=0,
    project="runs/train",
    name="dietary_yolov8s",
    exist_ok=True,
    workers=4,
    verbose=True
)

In [ ]:
best_model = YOLO("runs/train/dietary_yolov8s/weights/best.pt")

metrics = best_model.val(
    data=DATA_YAML,
    split="val",
    imgsz=640,
    device=0
)

print(metrics)

In [ ]:

df = pd.DataFrame({
    "Class": metrics.names.values(),
    "Precision": metrics.box.p,
    "Recall": metrics.box.r,
    "mAP50": metrics.box.map50,
    "mAP50-95": metrics.box.map
})

df.sort_values("mAP50", ascending=False)


In [ ]:
TEST_IMAGES = f"{DATASET_PATH}/test/images"

results = best_model.predict(
    source=TEST_IMAGES,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    save=True,
    device=0
)


In [ ]:
best_model.val(
    data=DATA_YAML,
    plots=True


In [ ]:
shutil.copy(
    "runs/train/dietary_yolov8s/weights/best.pt",
    "dietary_food_detector.pt"
)

In [ ]:
best_model.export(format="torchscript")
best_model.export(format="onnx")